<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Recalage par recherche des RBS

Status:
  * code devrait être OK
  * mais reste à trouver les données
  * et à rédiger

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

In [ ]:
def next_start_codon_after_rbs(arn, index, rbs):
    """
    recherche le premier start après une séquence RBS
    renvoie l'index du START, ou None si on n'en trouve plus
    """
    # on cherche le premier RBS
    index = arn.find(rbs, index)
    if index <= 1:
        return None
    index = arn.find(start_codon, index)
    if index <= 1:
        return None
    return index

In [ ]:
# avec par défaut une longueur minimale de 300
def regions_codantes_une_phase_rbs(adn, phase, rbs, longueur_minimale=300):
    # on initialise index à la phase; avec next_start_codon
    # et next_stop_codon on reste toujours sur la même phase
    index = phase
    # les résultats sont retournés sous la forme d'une liste 
    # de couples [start_gene, stop_gene]
    genes = []
    # la boucle 
    while True:      # boucle principale
        # on cherche le premier STOP à partir de index
        stop1 = next_stop_codon(adn, index)
        # s'il n'y a plus de stop, on a fini
        if not stop1:
            return genes
        # sinon on cherche un deuxieme stop
        stop2 = stop1 + 3
        # mais il faut qu'il soit assez loin
        while True:  # boucle interne
            stop2 = next_stop_codon(adn, stop2)
            # s'il n'y en a pas on a fini
            if not stop2:
                return genes
            # s'il est assez loin, on peut sortir de la boucle interne
            if stop2 - stop1 >= longueur_minimale:
                break
            # sinon il faut continuer à chercher un STOP
            # pour passer au stop suivant et ne pas retomber a nouveau sur stop2
            stop2 += 3
        # à ce stade on a trouvé un ORF, reste à trouver le bon start
        ##########
        # c'est ici la seule différence avec la version simple
        # de regions_codantes_une_phase
        start = next_start_codon_after_rbs(adn, stop1, rbs)
        ##########
        # s'il n'y en a pas: c'est qu'on ne trouvera plus rien
        # on a fini
        if not start:
            return genes
        # ici c'est un peu subtil
        # tout d'abord rien ne nous dit qu'il y a un codon START entre start1 et start2
        # et même s'il y en a un, rien de nous dit qu'on obtient un gêne assez long
        # donc on reteste la longueur du gêne trouvé et on ne garde que ceux 
        # qui sont assez longs - par rapport à longueur_minimale
        if stop2 - start < longueur_minimale:
            pass
        else:
            # cette fois on a trouvé un genes, on l'ajoute dans les résultats
            genes.append( [start, stop2] )
        # on peut passer à l'ORF suivant
        index = stop2 + 3

XXX manque un exemple de RBS